In [13]:
from functools import reduce
from pathlib import Path
import cv2
import pytesseract
from pytesseract import Output
import numpy as np
import pandas

In [14]:
tiff_dir = Path.cwd().parent / "collection_tiff_docs"
path_one_doc_sample: str = str(tiff_dir / "0000066686.tiff")
path_multi_doc_sample: str = str(tiff_dir / "memo.tiff")
with open(tiff_dir / "0000066686.tiff") as f:
    pass

In [15]:
one_doc_sample = cv2.imread(path_one_doc_sample, cv2.IMREAD_COLOR)
# ret, multi_doc_sample = cv2.imreadmulti(path_multi_doc_sample, [], cv2.IMREAD_COLOR)
ret, other_doc_sample = cv2.imreadmulti(path_one_doc_sample, [], cv2.IMREAD_COLOR)
...
cv2.imwrite("1_00001.tif", one_doc_sample)
# cv2.imwritemulti("1_00002.tif", multi_doc_sample)
# img1=cv2.imread("1_00001.tif")


True

In [16]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# https://muthu.co/all-tesseract-ocr-options/
custom_config = r'--oem 3 --psm 6'
result = pytesseract.image_to_string(one_doc_sample, config=custom_config)
result

'BROWN & WILLTAMSON TOBACCO CORPORATION\nRESEARCH, DEVELOPMENT & ENGINEERING -\nINTERNAL CORRESPONDENCE\nTO: Dr. W. H. Deines\nce: Mr. E. E. Kohnhorst\nMe. A. J. Mellman\nMr. L. E. Gravely\nDr. B. A. Sproule\nMr. M. L. Reynolds\nMr. T. F. Riehl\nMr. R. F. Brotzge\nMs. BL A. Kerr\nMr. T. E. Albert\nFROM: Ms. A. L. Kirby\nDATE: August 20, 198%\nSUBJECT: BRAND SUMMARY OF DUPONT PANELISTS/311\nI have attached a brand-by-brand tally of usable (21-59 years old)\nDupont panelists. This information was from early June.\nThe system is consistantly updated with brand changes of current\npanelists. Recently, 200 panelists per month have been added to the\nsystem.\nAnother tally will be available late-September.\nPam <8\nALK/sek ir}\nQa\n06741 S\na\nAttachments 2\n&\n<9\n&\n3 nN\n\x0c'

In [17]:
d_b = pytesseract.image_to_data(one_doc_sample, output_type=Output.DATAFRAME, config=custom_config)
text_before_preprocess = d_b["text"]

In [18]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,2)

#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((2,2),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

In [19]:
pr_one_doc_sample = reduce(lambda item, func: func(item),
        # [one_doc_sample, get_grayscale, remove_noise, thresholding, opening])
        # remove_noise - недостаточное разрешение изображения
        [one_doc_sample, get_grayscale, thresholding])

cv2.imwrite("1_00001_pr.tif", pr_one_doc_sample)

True

In [20]:
d = pytesseract.image_to_data(pr_one_doc_sample, output_type=Output.DATAFRAME, config=custom_config)
text_after_preprocess = d["text"]
[s for s in text_before_preprocess if s][:20], [s for s in text_after_preprocess if s][:20]

([nan,
  nan,
  nan,
  nan,
  'BROWN',
  '&',
  'WILLTAMSON',
  'TOBACCO',
  'CORPORATION',
  nan,
  'RESEARCH,',
  'DEVELOPMENT',
  '&',
  'ENGINEERING',
  '-',
  nan,
  'INTERNAL',
  'CORRESPONDENCE',
  nan,
  'TO:'],
 [nan,
  nan,
  nan,
  nan,
  'BROWN',
  '&',
  'WILLIAMSON',
  'TOBACCO',
  'CORPORATION',
  nan,
  'RESEARCH,',
  'DEVELOPMENT',
  '&',
  'ENGINEFRING',
  '~',
  nan,
  'INTERNAL',
  'CORRESPONDENCE',
  nan,
  'TO:'])

In [21]:
n_boxes = len(d['text'])
for i in range(n_boxes):
    if float(d['conf'][i]) > 0:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(pr_one_doc_sample, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imwrite('1_00001_pr_contours.tif', pr_one_doc_sample)

True

In [22]:
dd = d.where(d["conf"] > 40)
dd.dropna(subset=["conf"], inplace=True)
dd


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
4,5.0,1.0,1.0,1.0,1.0,1.0,238.0,165.0,37.0,9.0,86.148788,BROWN
5,5.0,1.0,1.0,1.0,1.0,2.0,283.0,165.0,6.0,9.0,87.333069,&
6,5.0,1.0,1.0,1.0,1.0,3.0,298.0,165.0,75.0,9.0,52.046848,WILLIAMSON
7,5.0,1.0,1.0,1.0,1.0,4.0,381.0,165.0,52.0,9.0,91.039345,TOBACCO
8,5.0,1.0,1.0,1.0,1.0,5.0,442.0,165.0,82.0,9.0,92.391891,CORPORATION
...,...,...,...,...,...,...,...,...,...,...,...,...
146,5.0,1.0,1.0,1.0,25.0,1.0,697.0,795.0,15.0,12.0,75.315247,a
148,5.0,1.0,1.0,1.0,26.0,1.0,120.0,808.0,36.0,9.0,51.852295,6741
151,5.0,1.0,1.0,1.0,27.0,1.0,696.0,821.0,16.0,12.0,89.573608,a
153,5.0,1.0,1.0,1.0,28.0,1.0,121.0,830.0,81.0,25.0,95.032150,Attachments
